<a href="https://colab.research.google.com/github/LeowYR/DSL_Homework/blob/main/Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/pe8sutd/eda4u &> /dev/null
!git clone https://github.com/pe8sutd/eda4u &> /dev/null
%load_ext plugin

**Hi Prof, I did my best to add meaninful parameters to the counter module and its testbench to genearalise it to be adaptable to multiple applications, and have gotten the testbench to output the monitor and .vcd and waveform successfully. I also have documented the various version of the code I went through, namely version 1, 2 and 3 , with 3 complete with upgrades and debugging.**

1. Useful commands: %%verilog, %%waveform, in Google Colab (a similar platform where you perform your homework).
2. Write Verilog source files (module and testbench), as per the description of the decade_counter as listed below, using if-else statements.
3. Submit ipynb (with Verilog HDL module and testbench) for assessment; (this file must be able to be run in Google Colab).
4. The specifications of the decade_counter are:
  - When "rst" is low, the output "count" and output "ten" are set to ‘1’ immediately and counting is disabled.
  - When "rst" is high, the circuit increments on every three positive edge of the clk.
  - The highest count that can be reached is 5'b10010, after which the count will restart from 5'b0 again.
  - Only when the count is 5'b10010, the output ten is set to ‘0’ else the output ten will always be set to ‘1’.
5. Write a Verilog test stimulus module for the decade_counter and verify its proper operation as follows:
  - Create a clock (Period: 2 unit time);
  - Test for reset active;
  - Then test for the counting for at least two complete count cycles ie: counting from reset value to 5'b10010 and also counting from '0';
  - Use Weveform Viewer for viewing the inputs and outputs;
  - You are free to propose other test to verify the funcationality of the module;

The Quality level for the Verilog module Coding:

1. Basic marks: direct implementation of the Verilog HDL at Structure Level or Behavioural Level without parameterized code.

2. Intermediate marks: implementation of the Verilog HDL with parameterized code or other concise coding style (function, generate, etc but must be synthesizable), e.g. parameterized the count number.

3. **Advanced marks: generalized Verilog HDL module**, e.g. a counter that can be count-up, count-down to a specific number.



There is no specific Testbench style in the question, as the testbench itself is an open question. The quality of the testbench depends on:

1. Basic marks: direct implementation using Truth Table style with limited coverage.

2. Intermediate marks: using behavioural testbench and self-checking if the results is correct, e.g. using if-else, for loop, case statement, random number etc. to have wide coverage range.

3. **Advanced marks: generalized testbench** that can incorporate Phyton, C or other coding for the system level design and verification.

Help me code a Verilog module and testbench that is generalized for multiple parameters, as per the description of the decade_counter as listed below, using if-else statements. Also include $monitor and output a vcd dumpfile for external testing and verification.



The specifications of the decade_counter are:
When "rst" is low, the output "count" and output "ten" are set to ‘1’ immediately and counting is disabled.
When "rst" is high, the circuit increments on every three positive edge of the clk.
The highest count that can be reached is 5'b10010, after which the count will restart from 5'b0 again.
Only when the count is 5'b10010, the output ten is set to ‘0’ else the output ten will always be set to ‘1’.
Write a Verilog test stimulus module for the decade_counter and verify its proper operation as follows:
Create a clock (Period: 2 unit time);
Test for reset active;
Then test for the counting for at least two complete count cycles ie: counting from reset value to 5'b10010 and also counting from '0';
Use Weveform Viewer for viewing the inputs and outputs;


Propose other ways to make the module more generalized to be adaptable to more situations, other tests to verify the existing and new funcationality of the module

In [5]:
%%verilog
//version 1


module decade_counter(
    input wire clk,           // Clock input
    input wire rst,           // Active low reset
    output reg [4:0] count,   // 5-bit counter output
    output reg ten            // Output that goes low when count reaches 10010
);

    // Internal counter to track every third clock edge
    reg [1:0] clk_counter;

    // Define the maximum count value (18 in decimal, 10010 in binary)
    localparam MAX_COUNT = 5'b10010;

    always @(posedge clk or negedge rst) begin
        if (!rst) begin
            // Reset condition (active low)
            count <= 5'b00001;    // Initialize count to 1
            ten <= 1'b1;          // Initialize ten to 1
            clk_counter <= 2'b00; // Reset clock edge counter
        end
        else begin
            // Increment clk_counter on every clock edge
            clk_counter <= clk_counter + 1'b1;

            // Only increment count on every third clock edge
            if (clk_counter == 2'b10) begin
                // Reset clk_counter for next cycle
                clk_counter <= 2'b00;

                // Check if maximum count reached
                if (count == MAX_COUNT) begin
                    count <= 5'b00000;  // Reset count to 0
                    ten <= 1'b1;        // Set ten back to 1
                end
                else begin
                    count <= count + 1'b1;  // Increment count

                    // Check if count is about to become 10010
                    if (count + 1'b1 == MAX_COUNT) begin
                        ten <= 1'b0;    // Set ten to 0
                    end
                    else begin
                        ten <= 1'b1;    // Keep ten at 1
                    end
                end
            end
        end
    end

endmodule


`timescale 1us/100ns

module decade_counter_tb;
    // Testbench signals
    reg clk;
    reg rst;
    wire [4:0] count;
    wire ten;

    // Instantiate the decade_counter module
    decade_counter dut (
        .clk(clk),
        .rst(rst),
        .count(count),
        .ten(ten)
    );

    // Clock generation with period of 2 time units
    always #1 clk = ~clk;

    // Variable to count cycles for verification
    integer cycle_count = 0;

    // Setup monitoring
    initial begin
        $monitor("Time=%0t, rst=%b, clk=%b, count=%b (%d), ten=%b, cycle_count=%0d",
                 $time, rst, clk, count, count, ten, cycle_count);
    end

    // Setup VCD dump file
    initial begin
        $dumpfile("decade_counter.vcd");
        $dumpvars(0, decade_counter_tb);
    end

    // Test sequence
    initial begin
        // Initialize signals
        clk = 0;
        rst = 0;  // Start with reset active (active low)

        // Hold in reset for a few clock cycles
        #10;

        // Release reset
        rst = 1;

        // Count cycles to ensure we test for at least two complete count cycles
        // The counter increments every 3 clock edges
        // Each complete cycle goes from 0 to 18 (19 total values)
        // So we need 19 * 3 * 2 = 114 clock edges for two complete cycles
        // With a clock period of 2, we need at least 228 time units
        #300;

        // Apply reset again to test reset functionality
        rst = 0;
        #10;

        // Release reset and count for another cycle
        rst = 1;
        #150;

        // End simulation
        $display("Simulation complete!");
        $finish;
    end

    // Track count cycles for verification
    always @(posedge clk) begin
        if (rst && count == 5'b00000) begin
            cycle_count = cycle_count + 1;
        end
    end

endmodule

VCD info: dumpfile decade_counter.vcd opened for output.
Time=0, rst=0, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=10, rst=0, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=20, rst=0, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=30, rst=0, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=40, rst=0, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=50, rst=0, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=60, rst=0, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=70, rst=0, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=80, rst=0, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=90, rst=0, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=100, rst=1, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=110, rst=1, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=120, rst=1, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=130, rst=1, clk=1, count=00001 ( 1), ten=1, cycle_count=0
Time=140, rst=1, clk=0, count=00001 ( 1), ten=1, cycle_count=0
Time=150,

In [6]:
%%verilog
// version 2

// Decade counter module
// Specifications:
// - When rst is low, output count and ten are set to 1 immediately and counting is disabled
// - When rst is high, the circuit increments on every third positive edge of clk
// - Highest count is 5'b10010 (18 in decimal), after which count restarts from 5'b0
// - Output ten is set to 0 only when count is 5'b10010, otherwise ten is 1

`timescale 1us/100ns

module decade_counter #(
    parameter COUNT_MAX = 5'b10010,  // Maximum count value (18 in decimal)
    parameter COUNT_WIDTH = 5         // Width of the count output
)(
    input wire clk,                  // Clock input
    input wire rst,                  // Active low reset
    output reg [COUNT_WIDTH-1:0] count, // Counter output
    output reg ten                   // Ten output (0 when count reaches maximum)
);

    // Internal counter to track every third clock edge
    reg [1:0] clk_counter;

    // Main counter logic
    always @(posedge clk or negedge rst) begin
        if (!rst) begin
            // Reset condition (active low)
            count <= 1;              // Set count to 1 when reset is active
            ten <= 1;                // Set ten to 1 when reset is active
            clk_counter <= 0;        // Reset the clock edge counter
        end else begin
            // Normal operation
            if (clk_counter == 2) begin  // Check if it's the third clock edge
                clk_counter <= 0;        // Reset clock edge counter

                // Increment or reset the main counter
                if (count == COUNT_MAX) begin
                    count <= 0;          // Restart from 0 after reaching maximum
                    ten <= 1;            // Set ten to 1 for all counts except maximum
                end else begin
                    count <= count + 1;  // Increment the counter

                    // Check if the next count will be maximum
                    if (count + 1 == COUNT_MAX) begin
                        ten <= 0;        // Set ten to 0 when count reaches maximum
                    end else begin
                        ten <= 1;        // Set ten to 1 for all other counts
                    end
                end
            end else begin
                clk_counter <= clk_counter + 1;  // Increment clock edge counter
            end
        end
    end

endmodule

// Testbench for decade counter
`timescale 1us/100ns

module decade_counter_tb;
    // Parameters for testbench
    parameter COUNT_MAX = 5'b10010;  // Maximum count value (18 in decimal)
    parameter COUNT_WIDTH = 5;       // Width of the count output

    // Testbench signals
    reg clk;
    reg rst;
    wire [COUNT_WIDTH-1:0] count;
    wire ten;

    // Instantiate the decade counter
    decade_counter #(
        .COUNT_MAX(COUNT_MAX),
        .COUNT_WIDTH(COUNT_WIDTH)
    ) dut (
        .clk(clk),
        .rst(rst),
        .count(count),
        .ten(ten)
    );

    // Clock generation (period = 2 time units)
    always begin
        #1 clk = ~clk;  // Toggle clock every 1 time unit
    end

    // Monitor signals
    initial begin
        $monitor("Time=%0t, rst=%b, clk=%b, count=%b (%0d), ten=%b, clk_counter=%b",
                 $time, rst, clk, count, count, ten, dut.clk_counter);
    end

    // Generate VCD file for waveform viewing
    initial begin
        $dumpfile("decade_counter_tb.vcd");
        $dumpvars(0, decade_counter_tb);
    end

    // Test sequence
    initial begin
        // Initialize signals
        clk = 0;
        rst = 0;  // Start with reset active (active low)

        // Test reset condition
        #4 rst = 1;  // Release reset

        // Wait for at least two complete count cycles
        // Cycle 1: count from reset value (1) to COUNT_MAX
        // Cycle 2: count from 0 to COUNT_MAX
        // Since we increment on every third clock edge, we need 3 clock edges per count
        // For first cycle: (COUNT_MAX - 1) counts * 3 clock edges * 2 time units per clock cycle
        // For second cycle: (COUNT_MAX + 1) counts * 3 clock edges * 2 time units per clock cycle

        // Wait for first cycle plus some extra time
        #((COUNT_MAX - 1) * 3 * 2 + 10);

        // Test reset during counting
        rst = 0;  // Assert reset
        #4 rst = 1;  // Release reset

        // Wait for second complete cycle
        #((COUNT_MAX + 1) * 3 * 2 + 10);

        // End simulation
        $display("Simulation completed successfully");
        $finish;
    end

endmodule

VCD info: dumpfile decade_counter_tb.vcd opened for output.
Time=0, rst=0, clk=0, count=00001 (1), ten=1, clk_counter=00
Time=10, rst=0, clk=1, count=00001 (1), ten=1, clk_counter=00
Time=20, rst=0, clk=0, count=00001 (1), ten=1, clk_counter=00
Time=30, rst=0, clk=1, count=00001 (1), ten=1, clk_counter=00
Time=40, rst=1, clk=0, count=00001 (1), ten=1, clk_counter=00
Time=50, rst=1, clk=1, count=00001 (1), ten=1, clk_counter=01
Time=60, rst=1, clk=0, count=00001 (1), ten=1, clk_counter=01
Time=70, rst=1, clk=1, count=00001 (1), ten=1, clk_counter=10
Time=80, rst=1, clk=0, count=00001 (1), ten=1, clk_counter=10
Time=90, rst=1, clk=1, count=00010 (2), ten=1, clk_counter=00
Time=100, rst=1, clk=0, count=00010 (2), ten=1, clk_counter=00
Time=110, rst=1, clk=1, count=00010 (2), ten=1, clk_counter=01
Time=120, rst=1, clk=0, count=00010 (2), ten=1, clk_counter=01
Time=130, rst=1, clk=1, count=00010 (2), ten=1, clk_counter=10
Time=140, rst=1, clk=0, count=00010 (2), ten=1, clk_counter=10
Time=1

In [4]:
%%verilog
// version 3,
//upgraded to be module to generalized to be adaptable for multiple use cases
//upgraded testbench to be generalised to test the multiple use cases

// Enhanced configurable counter module with additional parameterization
// Features:
// - Configurable reset value, maximum count, and increment step
// - Selectable clock edge triggering (positive or negative)
// - Configurable enable signal to pause/resume counting
// - Multiple output flags for different count conditions
// - Configurable increment frequency (every Nth clock edge)

`timescale 1us/100ns

module configurable_counter #(
    parameter COUNT_MAX = 5'b10010,     // Maximum count value (default: 18 decimal)
    parameter COUNT_WIDTH = 5,          // Width of the count output (default: 5 bits)
    parameter RESET_VALUE = 1,          // Value after reset (default: 1)
    parameter INCREMENT_STEP = 1,       // How much to increment by (default: 1)
    parameter CLK_DIVIDER = 3,          // Increment after every Nth clock edge (default: 3)
    parameter CLK_DIV_WIDTH = $clog2(CLK_DIVIDER), // Width of clock divider counter
    parameter ACTIVE_LOW_RESET = 1,     // 1 for active low reset, 0 for active high
    parameter POSEDGE_TRIGGER = 1       // 1 for positive edge triggering, 0 for negative edge
)(
    input wire clk,                     // Clock input
    input wire rst,                     // Reset signal
    input wire enable,                  // Enable signal to control counting
    input wire [COUNT_WIDTH-1:0] load_value, // Value to load when load is asserted
    input wire load,                    // Signal to load a specific value
    output reg [COUNT_WIDTH-1:0] count, // Counter output
    output reg ten,                     // Output flag (configurable condition)
    output reg halfway,                 // Additional flag for halfway point
    output reg quarter,                 // Additional flag for quarter point
    output wire terminal_count          // Flag for reaching maximum count
);

    // Internal signals
    reg [CLK_DIV_WIDTH-1:0] clk_counter;
    wire reset_active = ACTIVE_LOW_RESET ? !rst : rst;

    // Terminal count indicator
    assign terminal_count = (count == COUNT_MAX);

    // Determine which clock edge to trigger on
    generate
        if (POSEDGE_TRIGGER) begin : pos_edge_trigger
            always @(posedge clk or posedge reset_active) begin
                if (reset_active) begin
                    count <= RESET_VALUE;
                    ten <= 1;
                    halfway <= (RESET_VALUE >= COUNT_MAX/2) ? 1 : 0;
                    quarter <= (RESET_VALUE >= COUNT_MAX/4) ? 1 : 0;
                    clk_counter <= 0;
                end else begin
                    counter_logic();
                end
            end
        end else begin : neg_edge_trigger
            always @(negedge clk or posedge reset_active) begin
                if (reset_active) begin
                    count <= RESET_VALUE;
                    ten <= 1;
                    halfway <= (RESET_VALUE >= COUNT_MAX/2) ? 1 : 0;
                    quarter <= (RESET_VALUE >= COUNT_MAX/4) ? 1 : 0;
                    clk_counter <= 0;
                end else begin
                    counter_logic();
                end
            end
        end
    endgenerate

    // Counter logic task to avoid code duplication
    task counter_logic;
        begin
            if (load) begin
                // Load operation takes precedence
                count <= load_value;
                ten <= (load_value == COUNT_MAX) ? 0 : 1;
                halfway <= (load_value >= COUNT_MAX/2) ? 1 : 0;
                quarter <= (load_value >= COUNT_MAX/4) ? 1 : 0;
                clk_counter <= 0;
            end else if (enable) begin
                // Only count when enabled
                if (clk_counter == CLK_DIVIDER-1) begin
                    clk_counter <= 0;

                    if (count == COUNT_MAX) begin
                        count <= 0;
                        ten <= 1;
                        halfway <= 0;
                        quarter <= 0;
                    end else begin
                        // Increment by the specified step
                        count <= count + INCREMENT_STEP;

                        // Update flags based on new count value
                        if (count + INCREMENT_STEP == COUNT_MAX) begin
                            ten <= 0;
                        end else begin
                            ten <= 1;
                        end

                        // Halfway flag
                        if (count + INCREMENT_STEP >= COUNT_MAX/2) begin
                            halfway <= 1;
                        end else begin
                            halfway <= 0;
                        end

                        // Quarter flag
                        if (count + INCREMENT_STEP >= COUNT_MAX/4) begin
                            quarter <= 1;
                        end else begin
                            quarter <= 0;
                        end
                    end
                end else begin
                    clk_counter <= clk_counter + 1;
                end
            end
        end
    endtask

endmodule

// Comprehensive testbench for enhanced configurable counter
`timescale 1us/100ns

module configurable_counter_tb;
    // Testbench parameters
    parameter COUNT_MAX = 5'b10010;       // Maximum count (18 decimal)
    parameter COUNT_WIDTH = 5;            // Counter width
    parameter RESET_VALUE = 1;            // Reset value
    parameter INCREMENT_STEP = 1;         // Increment step
    parameter CLK_DIVIDER = 3;            // Clock divider

    // Test configurations to verify
    parameter NUM_CONFIGS = 3;            // Number of different configurations to test

    // Testbench signals
    reg clk;
    reg rst;
    reg enable;
    reg [COUNT_WIDTH-1:0] load_value;
    reg load;
    wire [COUNT_WIDTH-1:0] count;
    wire ten;
    wire halfway;
    wire quarter;
    wire terminal_count;

    // Configuration storage
    reg [3:0] active_config;
    reg [31:0] config_params [0:NUM_CONFIGS-1][0:4]; // [config_idx][param_idx]

    // DUT instantiation with default parameters
    configurable_counter #(
        .COUNT_MAX(COUNT_MAX),
        .COUNT_WIDTH(COUNT_WIDTH),
        .RESET_VALUE(RESET_VALUE),
        .INCREMENT_STEP(INCREMENT_STEP),
        .CLK_DIVIDER(CLK_DIVIDER),
        .ACTIVE_LOW_RESET(1),
        .POSEDGE_TRIGGER(1)
    ) dut (
        .clk(clk),
        .rst(rst),
        .enable(enable),
        .load_value(load_value),
        .load(load),
        .count(count),
        .ten(ten),
        .halfway(halfway),
        .quarter(quarter),
        .terminal_count(terminal_count)
    );

    // Clock generation
    always begin
        #1 clk = ~clk;
    end

    // Monitor
    initial begin
        $monitor("Time=%0t, Config=%0d, rst=%b, enable=%b, load=%b, load_val=%d, count=%d, ten=%b, halfway=%b, quarter=%b, term=%b",
                 $time, active_config, rst, enable, load, load_value, count, ten, halfway, quarter, terminal_count);
    end

    // VCD file generation
    initial begin
        $dumpfile("configurable_counter_tb.vcd");
        $dumpvars(0, configurable_counter_tb);
    end

    // Define test configurations
    initial begin
        // Config 0: Original specification (for backward compatibility)
        config_params[0][0] = COUNT_MAX;      // COUNT_MAX
        config_params[0][1] = RESET_VALUE;    // RESET_VALUE
        config_params[0][2] = INCREMENT_STEP; // INCREMENT_STEP
        config_params[0][3] = CLK_DIVIDER;    // CLK_DIVIDER
        config_params[0][4] = 0;              // Test duration multiplier

        // Config 1: Faster counting (increment every clock cycle)
        config_params[1][0] = COUNT_MAX;      // COUNT_MAX
        config_params[1][1] = 0;              // RESET_VALUE (start from 0)
        config_params[1][2] = 1;              // INCREMENT_STEP
        config_params[1][3] = 1;              // CLK_DIVIDER (every clock cycle)
        config_params[1][4] = 1;              // Test duration multiplier

        // Config 2: Larger step size (count by 2)
        config_params[2][0] = COUNT_MAX;      // COUNT_MAX
        config_params[2][1] = 0;              // RESET_VALUE
        config_params[2][2] = 2;              // INCREMENT_STEP (count by 2)
        config_params[2][3] = 2;              // CLK_DIVIDER (every other clock)
        config_params[2][4] = 2;              // Test duration multiplier
    end

    // Task to run a single test configuration
    task run_test_config;
        input [3:0] config_id;
        input [31:0] max_count;
        input [31:0] reset_val;
        input [31:0] inc_step;
        input [31:0] clk_div;
        input [31:0] duration_mult;
        integer cycle_time;
        begin
            active_config = config_id;

            // Calculate cycle time based on parameters
            cycle_time = ((max_count + 1) * clk_div * 2) / inc_step;

            $display("\n-----------------------------------------------------");
            $display("Starting Test Configuration %0d", config_id);
            $display("COUNT_MAX=%0d, RESET_VALUE=%0d, INCREMENT_STEP=%0d, CLK_DIVIDER=%0d",
                     max_count, reset_val, inc_step, clk_div);
            $display("-----------------------------------------------------\n");

            // Test 1: Basic counting and overflow
            enable = 1;
            load = 0;
            rst = 0; // Active reset
            #4 rst = 1; // Release reset

            // Wait for full cycle plus some margin
            #(cycle_time + 10);

            // Test 2: Load functionality
            load_value = max_count / 2;
            load = 1;
            #2 load = 0;

            // Continue counting from loaded value
            #(cycle_time / 2 + 10);

            // Test 3: Enable/disable functionality
            enable = 0; // Pause counting
            #10;
            enable = 1; // Resume counting

            // Wait for remaining cycle
            #(cycle_time / 2 + 10);

            // Test 4: Reset during counting
            rst = 0; // Active reset
            #4 rst = 1; // Release reset

            // Wait for another complete cycle
            #(cycle_time * duration_mult);

            $display("\n-----------------------------------------------------");
            $display("Completed Test Configuration %0d", config_id);
            $display("-----------------------------------------------------\n");
        end
    endtask

    // Main test sequence
    initial begin
        // Initialize signals
        clk = 0;
        rst = 1; // Inactive reset
        enable = 0;
        load = 0;
        load_value = 0;
        active_config = 0;

        // Brief settling time
        #2;

        // Run all test configurations
        for (integer i = 0; i < NUM_CONFIGS; i = i + 1) begin
            run_test_config(
                i,
                config_params[i][0], // max_count
                config_params[i][1], // reset_val
                config_params[i][2], // inc_step
                config_params[i][3], // clk_div
                config_params[i][4] + 1 // duration_mult (add 1 to avoid zero)
            );
        end

        // End simulation
        $display("All tests completed successfully!");
        $finish;
    end

endmodule

Installing. Please wait... done!
VCD info: dumpfile configurable_counter_tb.vcd opened for output.
Time=0, Config=0, rst=1, enable=0, load=0, load_val= 0, count= x, ten=x, halfway=x, quarter=x, term=x

-----------------------------------------------------
Starting Test Configuration 0
COUNT_MAX=18, RESET_VALUE=1, INCREMENT_STEP=1, CLK_DIVIDER=3
-----------------------------------------------------

Time=20, Config=0, rst=0, enable=1, load=0, load_val= 0, count= 1, ten=1, halfway=0, quarter=0, term=0
Time=60, Config=0, rst=1, enable=1, load=0, load_val= 0, count= 1, ten=1, halfway=0, quarter=0, term=0
Time=110, Config=0, rst=1, enable=1, load=0, load_val= 0, count= 2, ten=1, halfway=0, quarter=0, term=0
Time=170, Config=0, rst=1, enable=1, load=0, load_val= 0, count= 3, ten=1, halfway=0, quarter=0, term=0
Time=230, Config=0, rst=1, enable=1, load=0, load_val= 0, count= 4, ten=1, halfway=0, quarter=1, term=0
Time=290, Config=0, rst=1, enable=1, load=0, load_val= 0, count= 5, ten=1, halfw

In [5]:
%%waveform configurable_counter_tb.vcd
#results with waveform
sign_list = [ 'configurable_counter_tb.clk', 'configurable_counter_tb.rst', 'configurable_counter_tb.enable', 'configurable_counter_tb.load', 'configurable_counter_tb.load_value', 'configurable_counter_tb.count', 'configurable_counter_tb.ten', 'configurable_counter_tb.halfway', 'configurable_counter_tb.quarter', 'configurable_counter_tb.terminal_count', 'configurable_counter_tb.active_config' ]
time_begin = 0
time_end = 900  # Adjusted shorter as full runtime was taking too long to submit, below code has full runtime for you to run
base = 'bin'

In [ ]:
%%waveform configurable_counter_tb.vcd
#results with waveform
sign_list = ['configurable_counter_tb.clk', 'configurable_counter_tb.rst', 'configurable_counter_tb.enable', 'configurable_counter_tb.load', 'configurable_counter_tb.load_value', 'configurable_counter_tb.count', 'configurable_counter_tb.ten', 'configurable_counter_tb.halfway', 'configurable_counter_tb.quarter', 'configurable_counter_tb.terminal_count', 'configurable_counter_tb.active_config']
time_begin = 0
time_end = 9000
base = 'dec' #to be easy to read